# Stage 2a: Generate Alpha Factors using Stock Universe

- Compute custom apha factors
- Compute univeral quant factors
- Compute date information
- Save Alpha Factors

In [1]:
from platform import python_version
import time
from datetime import datetime
import os
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 8)

# Set the import path for the tools directiory
import sys
# insert at position 1 in the path, as 0 is the path of this file.
sys.path.insert(1, '../tools')
import importlib
import ameritrade_functions as amc
importlib.reload(amc)
import utils
importlib.reload(utils)

print(f'Python version: {python_version()}')
print(f'Pandas version: {pd.__version__}')

Python version: 3.8.10
Pandas version: 0.25.3


## Configure Ameritrade Information

Ameritrade credentials are stored in environment variables to keep from having unencrypted passwords stored on disk.

The module automatically masks the account numbers to protect the actual accounts. An Ameritrade user can have many investment accounts. We will be working with only one for this demonstration.

In [2]:
username = os.getenv('maiotradeuser')
password = os.getenv('maiotradepw')
client_id = os.getenv('maiotradeclientid')

masked_account_number = '#---5311'
account_portfolios_file_name = 'data/portfolio_data.csv'
portfolio_file_name = 'data/portfolio_' + masked_account_number[-4:] + '.csv'
price_histories_file_name = 'data/price_histories.csv'

# Price History data from Stage 1

In [3]:
price_histories = utils.read_price_histories(price_histories_file_name)
print(f'Date range for price histories: {price_histories.date.min().date()} to {price_histories.date.max().date()}')
close = utils.get_close_values(price_histories)
print(f'You have {len(close.columns)} stocks')
print(close.columns.to_list())
close.tail()

Date range for price histories: 2018-07-09 to 2021-07-09
You have 526 stocks
['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEI', 'AEP', 'AES', 'AFL', 'AIG', 'AIH', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'ALL', 'ALLE', 'ALXN', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANET', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APTV', 'ARE', 'ATO', 'ATVI', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZO', 'BA', 'BABA', 'BAC', 'BAX', 'BBY', 'BDX', 'BEN', 'BF.B', 'BIIB', 'BIO', 'BK', 'BKNG', 'BKR', 'BLK', 'BLL', 'BMY', 'BR', 'BRK.B', 'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CAN', 'CARR', 'CAT', 'CB', 'CBOE', 'CBRE', 'CCI', 'CCL', 'CDNS', 'CDW', 'CE', 'CERN', 'CF', 'CFG', 'CHD', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMA', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COG', 'COKE', 'CONN', 'COO', 'COP', 'COST', 'CPB', 'CPRT', 'CRL', 'CRM', 'CSCO', 'CSX', 'CTAS', 'CTLT', 'CTSH', 'CTVA', 'CTXS', 

ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WST,WU,WY,WYNN,XEL,XLNX,XOM,XRAY,XYL,ZKIN
date,,,,,,,,,,,,,,,,,,,,,
2021-07-02 00:00:00+00:00,148.82,21.48,211.05,139.96,115.17,116.62,317.02,118.04,304.84,593.07,...,365.74,23.26,34.31,121.50,66.82,144.56,63.17,63.89,120.89,3.9500
2021-07-06 00:00:00+00:00,148.98,21.01,208.70,142.02,115.73,115.42,326.71,118.82,305.40,596.90,...,368.99,22.96,33.97,116.90,67.07,144.17,61.37,63.09,120.46,3.9300
2021-07-07 00:00:00+00:00,149.49,20.31,209.44,144.57,116.75,115.04,334.53,119.87,309.61,605.77,...,372.66,23.23,34.84,112.91,67.69,137.44,60.41,63.63,120.67,3.6200
2021-07-08 00:00:00+00:00,148.83,20.35,207.51,143.24,116.35,114.44,325.04,119.26,309.15,605.95,...,373.11,22.94,35.04,111.66,67.50,133.77,60.14,61.91,119.19,3.6600
2021-07-09 00:00:00+00:00,150.03,20.89,213.47,145.11,116.58,115.13,324.77,119.74,312.62,604.50,...,374.19,23.31,35.82,113.87,67.67,135.59,61.23,62.42,121.05,3.7181


# Factors

### General
- Momentum
- Mean Reversion
- Overnight Sentiment

### Universal Quant Features
- Volatility 
- Dollar Volume
- Market Dispersion
- Market Volatility
- Date Parts

#TODO: SMA for market disp/vol

In [4]:
import trading_factors as alpha_factors
importlib.reload(alpha_factors)

all_factors = pd.concat(
[
    alpha_factors.FactorMomentum(price_histories, 252).demean().rank().zscore().for_al(),
    alpha_factors.FactorMeanReversion(price_histories, 5).demean().rank().zscore().smoothed().rank().zscore().for_al(),
    alpha_factors.OvernightSentiment(price_histories, 5).demean().rank().zscore().smoothed(10).rank().zscore().for_al(),
    alpha_factors.AnnualizedVolatility(price_histories, 20).rank().zscore().for_al(),
    alpha_factors.AnnualizedVolatility(price_histories, 120).rank().zscore().for_al(),
    alpha_factors.AverageDollarVolume(price_histories, 20).rank().zscore().for_al(),
    alpha_factors.AverageDollarVolume(price_histories, 120).rank().zscore().for_al(),
    alpha_factors.MarketDispersion(price_histories, 20).for_al(),
    alpha_factors.MarketDispersion(price_histories, 120).for_al(),
    alpha_factors.MarketVolatility(price_histories, 20).for_al(),
    alpha_factors.MarketVolatility(price_histories, 120).for_al()
], axis=1)
alpha_factors.FactorDateParts(all_factors)
all_factors.sort_index(inplace=True)
all_factors = all_factors.dropna()

all_factors.to_csv('data/all_factors.csv')

In [5]:
all_factors = pd.read_csv('data/all_factors.csv', parse_dates=['date'])
all_factors.set_index(['date', 'ticker'], inplace=True)
all_factors.sort_index(inplace=True)
all_factors.iloc[all_factors.index.get_level_values('ticker') == 'AAPL'].tail()

,,momentum_252_day_logret,mean_reversion_5_day_logret_smoothed,overnight_sentiment_5_day_smoothed,annualzed_volatility_20_day,annualzed_volatility_120_day,average_dollar_volume_20_day,average_dollar_volume_120_day,market_dispersion20_day,market_dispersion120_day,market_volatility20_day,market_volatility120_day,is_January,is_December,weekday,quarter,year,month_start,month_end,quarter_start,quarter_end
date,ticker,,,,,,,,,,,,,,,,,,,,
2021-07-02 00:00:00+00:00,AAPL,0.312823,-1.597046,-1.452159,0.457710,0.102079,1.71559,1.722175,0.018903,0.024271,0.079476,0.109587,0,0,4,3,2021,0,0,0,0
2021-07-06 00:00:00+00:00,AAPL,0.405024,-1.603632,-1.175558,0.398438,0.023050,1.71559,1.722175,0.018505,0.024235,0.085545,0.109971,0,0,1,3,2021,0,0,0,0
2021-07-07 00:00:00+00:00,AAPL,0.358924,-1.603632,-0.866027,0.523568,-0.055979,1.71559,1.722175,0.018571,0.024244,0.085392,0.109705,0,0,2,3,2021,0,0,0,0
2021-07-08 00:00:00+00:00,AAPL,0.312823,-1.623389,-0.773826,0.556496,-0.187694,1.71559,1.722175,0.018488,0.024193,0.090817,0.110835,0,0,3,3,2021,0,0,0,0
2021-07-09 00:00:00+00:00,AAPL,0.148180,-1.636560,-0.793584,0.767241,-0.273309,1.71559,1.722175,0.018004,0.024127,0.102366,0.111923,0,0,4,3,2021,0,0,0,0


In [6]:
all_factors.describe()

,momentum_252_day_logret,mean_reversion_5_day_logret_smoothed,overnight_sentiment_5_day_smoothed,annualzed_volatility_20_day,annualzed_volatility_120_day,average_dollar_volume_20_day,average_dollar_volume_120_day,market_dispersion20_day,market_dispersion120_day,market_volatility20_day,market_volatility120_day,is_January,is_December,weekday,quarter,year,month_start,month_end,quarter_start,quarter_end
count,2.656300e+05,2.656300e+05,2.656300e+05,2.656300e+05,2.656300e+05,2.656300e+05,2.656300e+05,265630.000000,265630.000000,265630.000000,265630.000000,265630.000000,265630.000000,265630.000000,265630.000000,265630.000000,265630.000000,265630.000000,265630.000000,265630.000000
mean,1.203720e-19,5.082375e-19,2.407441e-19,4.146148e-19,1.604960e-19,-1.203720e-19,3.343668e-19,0.022690,0.021974,0.170330,0.169373,0.079208,0.085149,2.017822,2.516832,2020.015842,0.041584,0.045545,0.011881,0.015842
std,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,0.007544,0.004666,0.158216,0.089087,0.270064,0.279103,1.397907,1.115249,0.706230,0.199637,0.208496,0.108352,0.124863
min,-1.728771e+00,-1.728769e+00,-1.728761e+00,-1.728761e+00,-1.728761e+00,-1.728761e+00,-1.728761e+00,0.013432,0.015794,0.038930,0.074311,0.000000,0.000000,0.000000,1.000000,2019.000000,0.000000,0.000000,0.000000,0.000000
25%,-8.660273e-01,-8.660273e-01,-8.660270e-01,-8.660273e-01,-8.660273e-01,-8.660273e-01,-8.660273e-01,0.017947,0.016712,0.084993,0.102277,0.000000,0.000000,1.000000,2.000000,2020.000000,0.000000,0.000000,0.000000,0.000000
50%,3.292878e-03,3.292878e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.020699,0.022402,0.121420,0.137057,0.000000,0.000000,2.000000,3.000000,2020.000000,0.000000,0.000000,0.000000,0.000000
75%,8.660273e-01,8.660273e-01,8.660270e-01,8.660273e-01,8.660273e-01,8.660273e-01,8.660273e-01,0.026709,0.024573,0.177673,0.217815,0.000000,0.000000,3.000000,4.000000,2021.000000,0.000000,0.000000,0.000000,0.000000
max,1.728771e+00,1.728769e+00,1.728761e+00,1.728769e+00,1.728769e+00,1.728769e+00,1.728769e+00,0.053386,0.031236,0.888565,0.367985,1.000000,1.000000,4.000000,4.000000,2021.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
all_factors.head()

momentum_252_day_logret  \
date                      ticker                            
2019-07-10 00:00:00+00:00 A                      0.549913   
                          AAL                   -1.103119   
                          AAP                    0.227210   
                          AAPL                   0.009879   
                          ABBV                  -1.366551   

                                  mean_reversion_5_day_logret_smoothed  \
date                      ticker                                         
2019-07-10 00:00:00+00:00 A                                  -1.346793   
                          AAL                                -0.694800   
                          AAP                                 0.418197   
                          AAPL                               -1.057019   
                          ABBV                                1.379722   

                                  overnight_sentiment_5_day_smoothed  \
date                      ticker                                       
2019-07-10 00:00:00+00:00 A                                 0.233794   
                          AAL                               0.767241   
                          AAP                              -0.602597   
                          AAPL                             -0.075736   
                          ABBV                             -0.628940   

                                  annualzed_volatility_20_day  \
date                      ticker                                
2019-07-10 00:00:00+00:00 A                          0.866031   
                          AAL                        1.122877   
                          AAP                        0.411612   
                          AAPL                       0.734315   
                          ABBV                       0.905546   

                                  annualzed_volatility_120_day  \
date                      ticker                                 
2019-07-10 00:00:00+00:00 A                          -0.714558   
                          AAL                         0.457712   
                          AAP                        -0.754073   
                          AAPL                        1.406065   
                          ABBV                       -1.089948   

                                  average_dollar_volume_20_day  \
date                      ticker                                 
2019-07-10 00:00:00+00:00 A                          -0.075737   
                          AAL                         0.431369   
                          AAP                         0.075737   
                          AAPL                        1.722183   
                          ABBV                        1.603639   

                                  average_dollar_volume_120_day  \
date                      ticker                                  
2019-07-10 00:00:00+00:00 A                            0.082322   
                          AAL                          0.576256   
                          AAP                          0.332582   
                          AAPL                         1.722183   
                          ABBV                         1.399480   

                                  market_dispersion20_day  \
date                      ticker                            
2019-07-10 00:00:00+00:00 A                      0.013883   
                          AAL                    0.013883   
                          AAP                    0.013883   
                          AAPL                   0.013883   
                          ABBV                   0.013883   

                                  market_dispersion120_day  \
date                      ticker                             
2019-07-10 00:00:00+00:00 A                       0.016321   
                          AAL                     0.016321   
                          AAP                     0.01